In [4]:
import numpy as np
from dataclasses import dataclass
from typing import Tuple

# =========================
# Heston + Carr–Madan (FFT)
# =========================

@dataclass(frozen=True)
class HestonParams:
    kappa: float
    theta: float
    sigma: float
    rho: float
    v0: float

def heston_cf(u: np.ndarray, T: float, S0: float, r: float, q: float, p: HestonParams) -> np.ndarray:
    """
    Characteristic function φ(u) = E_Q[exp(i u ln S_T)].
    Little Heston Trap parametrization. Vectorized in u.
    """
    u = np.asarray(u, dtype=np.complex128)
    i = 1j
    x0 = np.log(S0)

    # Parameters
    a = p.kappa * p.theta
    b = p.kappa - p.rho * p.sigma * i * u
    d = np.sqrt(b*b + (p.sigma**2) * (i*u + u*u))  # sqrt(b^2 + σ^2(u^2 + i u))
    g = (b - d) / (b + d)                           # |g|<1 in the "little trap" branch

    eDT = np.exp(-d * T)
    one_minus_g_eDT = 1 - g * eDT
    one_minus_g     = 1 - g

    # Numerical guards
    eps = 1e-15
    one_minus_g_eDT = np.where(np.abs(one_minus_g_eDT) < eps, eps, one_minus_g_eDT)
    one_minus_g     = np.where(np.abs(one_minus_g)     < eps, eps, one_minus_g)

    C = i*u*(r - q)*T + (a/(p.sigma**2)) * ((b - d)*T - 2.0*np.log(one_minus_g_eDT/one_minus_g))
    D = ((b - d)/(p.sigma**2)) * ((1.0 - eDT) / one_minus_g_eDT)
    return np.exp(C + D*p.v0 + i*u*x0)

def _simpson_weights(N: int) -> np.ndarray:
    """Simpson weights on an N-point uniform grid (N must be even)."""
    if N % 2 != 0:
        raise ValueError("N must be even for Simpson weights.")
    w = np.ones(N)
    w[1:N-1:2] = 4.0
    w[2:N-2:2] = 2.0
    return w

def heston_fft_calls(
    S0: float,
    T: float,
    r: float,
    q: float,
    p: HestonParams,
    N: int = 4096,         # even, power of two preferred
    eta: float = 0.25,     # frequency step Δv > 0
    alpha: float = 1.5,    # damping > 0
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Carr–Madan FFT for call prices across a log-strike grid k = ln K.
    Returns K (ascending) and C(K).
    """
    if N <= 0 or N % 2:
        raise ValueError("N must be a positive even integer.")
    if eta <= 0:
        raise ValueError("eta must be > 0.")
    if alpha <= 0:
        raise ValueError("alpha must be > 0.")

    i = 1j
    n = np.arange(N)
    v = eta * n  # frequency grid v_n = n Δv, includes 0

    # ψ(v) = e^{-rT} φ(v - i(α+1)) / [(α + iv)(α + iv + 1)]
    u_shift = v - (alpha + 1.0)*i
    phi_shift = heston_cf(u_shift, T, S0, r, q, p)

    denom = (alpha**2 + alpha - v**2 + i*(2*alpha + 1.0)*v)  # (α+iv)(α+iv+1)
    # guard at v=0 is unnecessary if alpha>0, but keep tiny floor
    denom = np.where(np.abs(denom) < 1e-15, 1e-15, denom)

    psi = np.exp(-r*T) * phi_shift / denom

    # Simpson weights for the v-integral
    w = _simpson_weights(N) * (eta / 3.0)

    # FFT coupling: k-grid spacing and shift
    lam = 2.0 * np.pi / (N * eta)   # Δk
    b   = 0.5 * N * lam             # half-width in k; k_j = -b + j Δk
    x   = psi * np.exp(1j * b * v) * w

    F   = np.fft.fft(x)
    F   = np.real(F)  # imaginary residuals are numerical noise

    j = np.arange(N)
    k = -b + j * lam                 # k = ln K
    K = np.exp(k)

    calls = np.exp(-alpha * k) / np.pi * F
    order = np.argsort(K)
    return K[order], np.maximum(calls[order], 0.0)

def heston_fft_call_price(
    S0: float, K: float, T: float, r: float, q: float, p: HestonParams,
    N: int = 4096, eta: float = 0.25, alpha: float = 1.5
) -> float:
    """Price a single call via FFT + monotone linear interpolation on K."""
    K_grid, C_grid = heston_fft_calls(S0, T, r, q, p, N=N, eta=eta, alpha=alpha)
    # clamp or interpolate
    if K <= K_grid[0]:
        return float(C_grid[0])
    if K >= K_grid[-1]:
        return float(C_grid[-1])
    idx = int(np.searchsorted(K_grid, K))
    x0, x1 = K_grid[idx-1], K_grid[idx]
    y0, y1 = C_grid[idx-1], C_grid[idx]
    w = (K - x0) / (x1 - x0)
    return float(y0 + w * (y1 - y0))

def heston_fft_put_price(
    S0: float, K: float, T: float, r: float, q: float, p: HestonParams,
    N: int = 4096, eta: float = 0.25, alpha: float = 1.5
) -> float:
    """Put via put–call parity."""
    C = heston_fft_call_price(S0, K, T, r, q, p, N=N, eta=eta, alpha=alpha)
    return float(C - S0*np.exp(-q*T) + K*np.exp(-r*T))


In [5]:
# parameters
S0, r, q, T = 100.0, 0.01, 0.00, 1.0
hp = HestonParams(kappa=1.5, theta=0.04, sigma=0.3, rho=-0.7, v0=0.04)

# full grid of calls
K, C = heston_fft_calls(S0, T, r, q, hp, N=4096, eta=0.25, alpha=1.5)

# single strikes
call_100 = heston_fft_call_price(S0, 100, T, r, q, hp)

call_100



8.07774347291964

In [6]:
# Simulate Heston price with given parameters
S0 = 100          # Initial stock price
K = 100           # Strike price
T = 1             # Time to maturity
r = 0.01          # Risk-free rate
v0 = 0.04         # Initial variance
kappa = 2         # Mean reversion speed
theta = 0.04      # Long-run variance
sigma = 0.3       # Volatility of variance
rho = -0.7        # Correlation
N = 16384         # Number of grid points
alpha = 1.5       # Damping factor

# Simulation parameters
n_paths = 10000   # Number of paths
n_steps = 252     # Number of time steps (daily)
dt = T/n_steps    # Time step size

# Initialize arrays for paths
S = np.zeros((n_paths, n_steps + 1))
v = np.zeros((n_paths, n_steps + 1))

# Set initial values
S[:, 0] = S0
v[:, 0] = v0

# Generate correlated random numbers
rng = np.random.default_rng()
dW1 = rng.normal(0, np.sqrt(dt), (n_paths, n_steps))
dW2 = rho * dW1 + np.sqrt(1 - rho**2) * rng.normal(0, np.sqrt(dt), (n_paths, n_steps))

# Euler-Maruyama simulation
for t in range(n_steps):
    # Ensure variance stays positive
    v[:, t] = np.maximum(v[:, t], 0)

    # Update stock price
    S[:, t+1] = S[:, t] * np.exp((r - 0.5*v[:, t])*dt + np.sqrt(v[:, t])*dW1[:, t])

    # Update variance
    v[:, t+1] = v[:, t] + kappa*(theta - v[:, t])*dt + sigma*np.sqrt(v[:, t])*dW2[:, t]

# Calculate call option prices
payoffs = np.maximum(S[:, -1] - K, 0)
discounted_payoffs = np.exp(-r*T) * payoffs
price = np.mean(discounted_payoffs)

# Calculate 95% confidence interval
std_error = np.std(discounted_payoffs) / np.sqrt(n_paths)
ci_lower = price - 1.96 * std_error
ci_upper = price + 1.96 * std_error

print(f"Heston call option price: {price:.8f}")
print(f"95% Confidence Interval: [{ci_lower:.8f}, {ci_upper:.8f}]")


Heston call option price: 8.21021002
95% Confidence Interval: [7.99336721, 8.42705284]


In [7]:
# Generate strike grid centered around S0
strikes = np.arange(S0-20, S0+25, 5)

# Get FFT prices for all strikes
K_fft, C_fft = heston_fft_calls(S0, T, r, q, hp, N=4096, eta=0.25, alpha=1.5)

# Interpolate FFT prices to our strike grid
fft_prices = np.interp(strikes, K_fft, C_fft)

# Calculate MC prices for all strikes
mc_prices = []
mc_ci_lower = []
mc_ci_upper = []

for K in strikes:
    payoffs = np.maximum(S[:, -1] - K, 0)
    discounted_payoffs = np.exp(-r*T) * payoffs
    price = np.mean(discounted_payoffs)

    # Calculate 95% confidence interval
    std_error = np.std(discounted_payoffs) / np.sqrt(n_paths)
    mc_prices.append(price)
    mc_ci_lower.append(price - 1.96 * std_error)
    mc_ci_upper.append(price + 1.96 * std_error)

# Print comparison table
print("\nPrice Comparison:")
print("Strike   FFT Price   MC Price   MC 95% CI")
print("-" * 45)
for i in range(len(strikes)):
    print(f"{strikes[i]:6.1f}   {fft_prices[i]:9.4f}   {mc_prices[i]:8.4f}   [{mc_ci_lower[i]:8.4f}, {mc_ci_upper[i]:8.4f}]")



Price Comparison:
Strike   FFT Price   MC Price   MC 95% CI
---------------------------------------------
  80.0     22.3804    22.4078   [ 22.0839,  22.7317]
  85.0     18.2617    18.3002   [ 17.9971,  18.6032]
  90.0     14.4595    14.5127   [ 14.2347,  14.7907]
  95.0     11.0425    11.1385   [ 10.8897,  11.3873]
 100.0      8.0777     8.2102   [  7.9934,   8.4271]
 105.0      5.6161     5.7817   [  5.5983,   5.9651]
 110.0      3.6817     3.8626   [  3.7124,   4.0128]
 115.0      2.2629     2.4459   [  2.3269,   2.5648]
 120.0      1.2993     1.4717   [  1.3806,   1.5627]
